## Final Ver.

In [4]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_absolute_error
import joblib

# --- Load Data ---
inventory = pd.read_csv("cleaned_Inventory.csv", parse_dates=["BALANCE_AS_OF_DATE"])
inbound = pd.read_csv("cleaned_Inbound.csv", parse_dates=["INBOUND_DATE"])
outbound = pd.read_csv("cleaned_Outbound.csv", parse_dates=["OUTBOUND_DATE"])
material = pd.read_csv("MaterialMaster.csv")

# --- Manual Capacity Dictionary (in KT) ---
manual_capacity = {
    "CHINA-WAREHOUSE": 70.0,
    "SINGAPORE-WAREHOUSE": 53.5
}

# --- Aggregate Monthly Data ---
inventory["Month"] = inventory["BALANCE_AS_OF_DATE"].dt.to_period("M").dt.to_timestamp()
inbound["Month"] = inbound["INBOUND_DATE"].dt.to_period("M").dt.to_timestamp()
outbound["Month"] = outbound["OUTBOUND_DATE"].dt.to_period("M").dt.to_timestamp()

# Convert units to KT
inventory["UNRESRICTED_STOCK_KT"] = inventory["UNRESRICTED_STOCK"] / 1_000_000  # KG → KT
inbound["Inbound_KT"] = inbound["NET_QUANTITY_MT"] / 1_000                     # MT → KT
outbound["Outbound_KT"] = outbound["NET_QUANTITY_MT"] / 1_000                 # MT → KT

# Aggregate sums per plant, material, month
inv_monthly = inventory.groupby(["PLANT_NAME", "MATERIAL_NAME", "Month"])["UNRESRICTED_STOCK_KT"].sum().reset_index()
inb_monthly = inbound.groupby(["PLANT_NAME", "MATERIAL_NAME", "Month"])["Inbound_KT"].sum().reset_index()
outb_monthly = outbound.groupby(["PLANT_NAME", "MATERIAL_NAME", "Month"])["Outbound_KT"].sum().reset_index()

# Merge all
merged = inv_monthly.merge(inb_monthly, on=["PLANT_NAME", "MATERIAL_NAME", "Month"], how="left")
merged = merged.merge(outb_monthly, on=["PLANT_NAME", "MATERIAL_NAME", "Month"], how="left")
merged = merged.merge(material, on="MATERIAL_NAME", how="left")
merged.fillna(0, inplace=True)

# --- Feature Engineering ---
merged = merged.sort_values(by=["PLANT_NAME", "MATERIAL_NAME", "Month"])

# Lag features from previous month
merged["Lag_Inv"] = merged.groupby(["PLANT_NAME", "MATERIAL_NAME"])["UNRESRICTED_STOCK_KT"].shift(1)
merged["Lag_Inb"] = merged.groupby(["PLANT_NAME", "MATERIAL_NAME"])["Inbound_KT"].shift(1)
merged["Lag_Outb"] = merged.groupby(["PLANT_NAME", "MATERIAL_NAME"])["Outbound_KT"].shift(1)

merged = merged.dropna(subset=["Lag_Inv", "Lag_Inb", "Lag_Outb"])

# Month number as cyclical features (to capture seasonality)
merged["Month_Number"] = merged["Month"].dt.month
merged["Month_sin"] = np.sin(2 * np.pi * merged["Month_Number"] / 12)
merged["Month_cos"] = np.cos(2 * np.pi * merged["Month_Number"] / 12)

# Features and target
features = ["Lag_Inv", "Lag_Inb", "Lag_Outb", "SHELF_LIFE_IN_MONTH", "Month_sin", "Month_cos"]
target = "UNRESRICTED_STOCK_KT"

X = merged[features]
y = merged[target]

# --- Train/Test Split ---
# Use last 20% of time as test (time-based split)
split_index = int(len(merged) * 0.8)
X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
y_train, y_test = y.iloc[:split_index], y.iloc[split_index:]

# --- Model Training with TimeSeriesSplit and GridSearch ---
tscv = TimeSeriesSplit(n_splits=5)
param_grid = {
    "n_estimators": [100, 200],
    "learning_rate": [0.05, 0.1],
    "max_depth": [3, 5]
}
gbr = GradientBoostingRegressor(random_state=42)
grid_search = GridSearchCV(gbr, param_grid, cv=tscv, scoring="neg_mean_absolute_error", n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_

# --- Evaluation ---
y_pred = best_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f"Best MAE on test set: {mae:.4f} KT")

# Save model for later use
joblib.dump(best_model, "monthly_inventory_forecast_model.pkl")

# --- Forecast next 3 months for each plant-material ---
latest = merged.groupby(["PLANT_NAME", "MATERIAL_NAME"]).tail(1).reset_index(drop=True)
forecast_months = pd.date_range(start=merged["Month"].max() + pd.offsets.MonthBegin(1), periods=3, freq='ME')

results = []

for _, row in latest.iterrows():
    lag_inv = row["Lag_Inv"]
    lag_inb = row["Lag_Inb"]
    lag_outb = row["Lag_Outb"]
    shelf_life = row["SHELF_LIFE_IN_MONTH"]
    plant = row["PLANT_NAME"]
    material = row["MATERIAL_NAME"]

    for month in forecast_months:
        month_num = month.month
        month_sin = np.sin(2 * np.pi * month_num / 12)
        month_cos = np.cos(2 * np.pi * month_num / 12)

        X_input = pd.DataFrame([{
            "Lag_Inv": lag_inv,
            "Lag_Inb": lag_inb,
            "Lag_Outb": lag_outb,
            "SHELF_LIFE_IN_MONTH": shelf_life,
            "Month_sin": month_sin,
            "Month_cos": month_cos
        }])

        pred = best_model.predict(X_input)[0]

        results.append({
            "PLANT_NAME": plant,
            "MATERIAL_NAME": material,
            "Forecast_Month": month,
            "Predicted_Inventory_KT": pred
        })

        lag_inv = pred  # update lag_inv for next forecast month
        # lag_inb and lag_outb can be updated if you have inbound/outbound forecast, else keep static

# Create forecast DataFrame
forecast_df = pd.DataFrame(results)

# Aggregate by plant and month to get total predicted inventory
plant_forecast = forecast_df.groupby(["PLANT_NAME", "Forecast_Month"])["Predicted_Inventory_KT"].sum().reset_index()
plant_forecast["Max_Capacity_KT"] = plant_forecast["PLANT_NAME"].map(manual_capacity)
plant_forecast["Utilization_%"] = (plant_forecast["Predicted_Inventory_KT"] / plant_forecast["Max_Capacity_KT"]) * 100

# Risk classification
def classify_risk(util):
    if util > 80:
        return "Overflow"
    elif util < 70:
        return "Underflow"
    else:
        return "Healthy"

plant_forecast["Warehouse_Risk"] = plant_forecast["Utilization_%"].apply(classify_risk)

# Save results for PowerBI
forecast_df.to_csv("monthly_inventory_forecast_per_product.csv", index=False)
plant_forecast.to_csv("monthly_warehouse_forecast_summary.csv", index=False)
merged.to_csv("merged_inventory_data.csv", index=False)

print("✅ Monthly forecast saved as 'monthly_inventory_forecast_per_product.csv' and 'monthly_warehouse_forecast_summary.csv'")


Fitting 5 folds for each of 8 candidates, totalling 40 fits
Best MAE on test set: 0.0998 KT
✅ Monthly forecast saved as 'monthly_inventory_forecast_per_product.csv' and 'monthly_warehouse_forecast_summary.csv'


In [327]:
plant_forecast

,PLANT_NAME,Forecast_Month,Predicted_Inventory_KT,Max_Capacity_KT,Utilization_%,Warehouse_Risk
0,CHINA-WAREHOUSE,2025-01-31,57.734623,70.0,82.478033,Overflow
1,CHINA-WAREHOUSE,2025-02-28,56.680148,70.0,80.971640,Overflow
2,CHINA-WAREHOUSE,2025-03-31,57.428136,70.0,82.040194,Overflow
3,SINGAPORE-WAREHOUSE,2025-01-31,44.918493,53.5,83.959799,Overflow
4,SINGAPORE-WAREHOUSE,2025-02-28,42.969693,53.5,80.317184,Overflow
5,SINGAPORE-WAREHOUSE,2025-03-31,42.043020,53.5,78.585084,Healthy


## Test ##

In [7]:
import pandas as pd
import numpy as np
import joblib
from sklearn.metrics import mean_absolute_error

print("Loading model and merged data...")
best_model = joblib.load("monthly_inventory_forecast_model.pkl")
merged = pd.read_csv("merged_inventory_data.csv", parse_dates=["Month"])

# Create Month_End column for actual data alignment
merged["Month_End"] = merged["Month"] + pd.offsets.MonthEnd(0)

# Recreate cyclical month features based on Month (month start)
merged["Month_Number"] = merged["Month"].dt.month
merged["Month_sin"] = np.sin(2 * np.pi * merged["Month_Number"] / 12)
merged["Month_cos"] = np.cos(2 * np.pi * merged["Month_Number"] / 12)

# Define backtest months as month-end dates
backtest_month_ends = pd.date_range("2024-08-31", "2024-12-31", freq="M")

all_results = []

for forecast_month_end in backtest_month_ends:
    print(f"\nForecasting for {forecast_month_end.strftime('%b %Y')}...")

    # Use month-start for feature extraction (previous month start before forecast month end)
    forecast_month_start = forecast_month_end - pd.offsets.MonthEnd(1) + pd.offsets.MonthBegin(1)

    train_data = merged[merged["Month"] < forecast_month_start].copy()
    latest = train_data.groupby(["PLANT_NAME", "MATERIAL_NAME"]).tail(1).reset_index(drop=True)

    month_num = forecast_month_start.month
    month_sin = np.sin(2 * np.pi * month_num / 12)
    month_cos = np.cos(2 * np.pi * month_num / 12)

    forecast_rows = []
    for _, row in latest.iterrows():
        X_input = pd.DataFrame([{
            "Lag_Inv": row["Lag_Inv"],
            "Lag_Inb": row["Lag_Inb"],
            "Lag_Outb": row["Lag_Outb"],
            "SHELF_LIFE_IN_MONTH": row["SHELF_LIFE_IN_MONTH"],
            "Month_sin": month_sin,
            "Month_cos": month_cos
        }])
        pred = best_model.predict(X_input)[0]
        forecast_rows.append({
            "PLANT_NAME": row["PLANT_NAME"],
            "MATERIAL_NAME": row["MATERIAL_NAME"],
            "Predicted_Inventory_KT": pred
        })

    forecast_df = pd.DataFrame(forecast_rows)

    # Aggregate predicted inventory per plant
    pred_agg = forecast_df.groupby("PLANT_NAME")["Predicted_Inventory_KT"].sum().reset_index()

    # Aggregate actual inventory per plant using Month_End to match snapshot
    actual_df = merged[merged["Month_End"] == forecast_month_end][
        ["PLANT_NAME", "UNRESRICTED_STOCK_KT"]
    ]
    actual_agg = actual_df.groupby("PLANT_NAME")["UNRESRICTED_STOCK_KT"].sum().reset_index()

    # Merge predicted and actual
    compare = pred_agg.merge(actual_agg, on="PLANT_NAME", how="inner")
    compare.rename(columns={"UNRESRICTED_STOCK_KT": "Actual_Inventory_KT"}, inplace=True)
    compare["Absolute_Error"] = (compare["Predicted_Inventory_KT"] - compare["Actual_Inventory_KT"]).abs()
    compare["Forecast_Month_End"] = forecast_month_end

    mae = compare["Absolute_Error"].mean()
    print(f"MAE for {forecast_month_end.strftime('%b %Y')}: {mae:.4f} KT")

    all_results.append(compare)

final_compare_df = pd.concat(all_results, ignore_index=True)

# MAE summary per month-end (average over plants)
mae_summary = final_compare_df.groupby("Forecast_Month_End")["Absolute_Error"].mean().reset_index()
mae_summary.columns = ["Month_End", "Mean Absolute Error (KT)"]

print("\nBacktest MAE Summary by Month-End:")
print(mae_summary.to_string(index=False))

# Save full results
final_compare_df.to_csv("backtest_late2024_plant_level_results.csv", index=False)
print("\nBacktest plant-level results saved to 'backtest_late2024_plant_level_results.csv'")


Loading model and merged data...

Forecasting for Aug 2024...
MAE for Aug 2024: 1.4346 KT

Forecasting for Sep 2024...


/var/folders/0b/vr0wp6rs19n6j8zhfrjmlh700000gn/T/ipykernel_3411/2587129071.py:19: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  backtest_month_ends = pd.date_range("2024-08-31", "2024-12-31", freq="M")


MAE for Sep 2024: 3.1579 KT

Forecasting for Oct 2024...
MAE for Oct 2024: 2.5182 KT

Forecasting for Nov 2024...
MAE for Nov 2024: 7.5908 KT

Forecasting for Dec 2024...
MAE for Dec 2024: 8.2741 KT

Backtest MAE Summary by Month-End:
 Month_End  Mean Absolute Error (KT)
2024-08-31                  1.434576
2024-09-30                  3.157937
2024-10-31                  2.518199
2024-11-30                  7.590787
2024-12-31                  8.274060

Backtest plant-level results saved to 'backtest_late2024_plant_level_results.csv'


In [8]:
final_compare_df

,PLANT_NAME,Predicted_Inventory_KT,Actual_Inventory_KT,Absolute_Error,Forecast_Month_End
0,CHINA-WAREHOUSE,67.866319,68.089208,0.222889,2024-08-31
1,SINGAPORE-WAREHOUSE,38.618967,41.265231,2.646264,2024-08-31
2,CHINA-WAREHOUSE,70.896951,65.107268,5.789683,2024-09-30
3,SINGAPORE-WAREHOUSE,42.089597,41.563405,0.526192,2024-09-30
4,CHINA-WAREHOUSE,66.377174,62.294444,4.082730,2024-10-31
5,SINGAPORE-WAREHOUSE,40.950343,39.996675,0.953668,2024-10-31
6,CHINA-WAREHOUSE,67.083260,54.434421,12.648839,2024-11-30
7,SINGAPORE-WAREHOUSE,42.562510,40.029775,2.532735,2024-11-30
8,CHINA-WAREHOUSE,83.199260,68.957550,14.241710,2024-12-31
9,SINGAPORE-WAREHOUSE,56.873586,54.567175,2.306411,2024-12-31


## Actual Inventory data

In [18]:
import pandas as pd

# Assuming 'inventory' DataFrame with columns:
# 'PLANT_NAME', 'MATERIAL_NAME', 'BALANCE_AS_OF_DATE', 'UNRESTRICTED_STOCK'

# Convert BALANCE_AS_OF_DATE to datetime if needed
inventory['BALANCE_AS_OF_DATE'] = pd.to_datetime(inventory['BALANCE_AS_OF_DATE'])

# Create 'Month' column as period or timestamp (month start)
inventory['Month'] = inventory['BALANCE_AS_OF_DATE'] + pd.offsets.MonthEnd(0)

# For each plant + material + month, find last snapshot date
last_snapshots = inventory.groupby(['PLANT_NAME', 'MATERIAL_NAME', 'Month'])['BALANCE_AS_OF_DATE'].max().reset_index()

# Merge to get inventory rows corresponding to last snapshot in each month
last_inventory = last_snapshots.merge(inventory, 
                                     on=['PLANT_NAME', 'MATERIAL_NAME', 'Month', 'BALANCE_AS_OF_DATE'],
                                     how='left')

# Now sum unrestricted stock per plant and month (sum over all materials)
monthly_total_inventory = last_inventory.groupby(['PLANT_NAME', 'Month'])['UNRESRICTED_STOCK'].sum().reset_index()

# Convert units if needed, for example kg to KT:
monthly_total_inventory['UNRESRICTED_STOCK_KT'] = monthly_total_inventory['UNRESRICTED_STOCK'] / 1_000_000

monthly_total_inventory = monthly_total_inventory.drop(columns='UNRESRICTED_STOCK')

monthly_total_inventory


,PLANT_NAME,Month,UNRESRICTED_STOCK_KT
0,CHINA-WAREHOUSE,2023-12-31,42.230173
1,CHINA-WAREHOUSE,2024-01-31,39.023693
2,CHINA-WAREHOUSE,2024-02-29,46.217648
3,CHINA-WAREHOUSE,2024-03-31,48.478708
4,CHINA-WAREHOUSE,2024-04-30,48.415015
5,CHINA-WAREHOUSE,2024-05-31,52.826486
6,CHINA-WAREHOUSE,2024-06-30,72.524391
7,CHINA-WAREHOUSE,2024-07-31,76.136124
8,CHINA-WAREHOUSE,2024-08-31,68.138683
9,CHINA-WAREHOUSE,2024-09-30,66.701368


In [30]:
# Ensure datetime types
df1 = monthly_total_inventory
df2 = plant_forecast
import pandas as pd
import numpy as np

# Step 1: Copy df1 and rename columns to match df2
df1_clean = df1.copy()
df1_clean["Month"] = pd.to_datetime(df1_clean["Month"])

# Use UNRESRICTED_STOCK_KT as predicted (historical actuals)
df1_clean["Predicted_Inventory_KT"] = df1_clean["UNRESRICTED_STOCK_KT"]

# Step 2: Assign Max_Capacity_KT per plant (from df2)
plant_capacity = df2.groupby("PLANT_NAME")["Max_Capacity_KT"].first().to_dict()
df1_clean["Max_Capacity_KT"] = df1_clean["PLANT_NAME"].map(plant_capacity)

# Step 3: Compute utilization
df1_clean["Utilization_%"] = (
    df1_clean["Predicted_Inventory_KT"] / df1_clean["Max_Capacity_KT"]
) * 100

# Step 4: Assign warehouse risk
def risk_label(util):
    if pd.isna(util):
        return np.nan
    if util >= 80:
        return "Overflow"
    elif util < 60:
        return "Underflow"
    else:
        return "Healthy"

df1_clean["Warehouse_Risk"] = df1_clean["Utilization_%"].apply(risk_label)

# Step 5: Rename Month column to match df2
df1_clean.rename(columns={"Month": "Forecast_Month"}, inplace=True)

# Step 6: Select and order columns like df2
df1_final = df1_clean[[
    "PLANT_NAME", "Forecast_Month", "Predicted_Inventory_KT",
    "Max_Capacity_KT", "Utilization_%", "Warehouse_Risk"
]]

# Step 7: Concatenate with df2
final_df = pd.concat([df1_final, df2], ignore_index=True).sort_values(
    by=["PLANT_NAME", "Forecast_Month"]
).reset_index(drop=True)

final_df = final_df.drop(columns=['Month_Start','Utilization_%'])
final_df.to_csv("Inventory_level_all.csv", index=False)


In [29]:
final_df

,PLANT_NAME,Forecast_Month,Predicted_Inventory_KT,Max_Capacity_KT,Warehouse_Risk
0,CHINA-WAREHOUSE,2023-12-31,42.230173,70.0,Healthy
1,CHINA-WAREHOUSE,2024-01-31,39.023693,70.0,Underflow
2,CHINA-WAREHOUSE,2024-02-29,46.217648,70.0,Healthy
3,CHINA-WAREHOUSE,2024-03-31,48.478708,70.0,Healthy
4,CHINA-WAREHOUSE,2024-04-30,48.415015,70.0,Healthy
5,CHINA-WAREHOUSE,2024-05-31,52.826486,70.0,Healthy
6,CHINA-WAREHOUSE,2024-06-30,72.524391,70.0,Overflow
7,CHINA-WAREHOUSE,2024-07-31,76.136124,70.0,Overflow
8,CHINA-WAREHOUSE,2024-08-31,68.138683,70.0,Overflow
9,CHINA-WAREHOUSE,2024-09-30,66.701368,70.0,Overflow
